In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

import numpy as np
import networkx as nx
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize

model = SentenceTransformer('all-MiniLM-L6-v2')

def textrank_summarize(text, num_sentences=3):
    # 1. Split article into sentences
    sentences = sent_tokenize(text)
    if len(sentences) <= num_sentences:
        return " ".join(sentences)

    # 2. Encode each sentence into a vector
    embeddings = model.encode(sentences)

    # 3. Create similarity matrix
    sim_matrix = np.zeros((len(sentences), len(sentences)))

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                # cosine similarity
                sim_matrix[i][j] = np.dot(embeddings[i], embeddings[j]) / (
                    np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
                )

    # 4. Build graph and apply PageRank
    graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(graph)

    # 5. Rank sentences by score
    ranked_sentences = sorted(
        ((scores[i], s) for i, s in enumerate(sentences)),
        reverse=True
    )

    # 6. Return top N sentences in original order
    selected = [s for _, s in ranked_sentences[:num_sentences]]
    return " ".join(selected)


[nltk_data] Downloading package punkt to C:\Users\Bashir
[nltk_data]     Rayaneh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package punkt_tab to C:\Users\Bashir
[nltk_data]     Rayaneh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [4]:
from datasets import load_dataset

ds = load_dataset("cnn_dailymail", "3.0.0")
article = ds["train"][0]["article"]

summary = textrank_summarize(article, num_sentences=3)
print(summary)


Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
